# <center id="p1"><h1> ⛓ `CHAINS` 🔗 - разбираемся с базовым элементом фрэймворка!</h1></center>

### Оглавление ноутбука
<img src='https://i.ytimg.com/vi/W3AoeMrg27o/maxresdefault.jpg' align="right" width="600" height="650" />
<br>

<p><font size="3" face="Arial" font-size="large"><ul type="square">
    
<li><a href="#p1">🚀 Введение </a></li>
<li><a href="#p2">💊 Generic Chains </a></li><ul type="square">
<li><a href="#p3">LLMChain 🔗 </a>
<li><a href="#p4">TransformChain 🤖 </a>
<li><a href="#p5">SequentialChain 🔗➕⛓</a></ul>
<li><a href="#plcel">LCEL - LangChain Expression Language 🤪🀄️</a>
<li><a href="#p6">Router Chain 🈯️ ➡️ 🇯🇵</a>
<li><a href="#p7">🛠 Utility Chains 🪝 </a></li>
<li><a href="#p8">🧸 Выводы и заключения ✅ </a></li>
    
</ul></font></p>

<div class="alert alert-info">
    
Как вы могли догадаться из названия, **цепочки (Chains)** - это один из фундаментальных строительных блоков фрэймворка `LangChain`. Они представляют собой просто цепочку компонентов, которые будут выполнены в определённом порядке. <br>

*Официальным определением цепочек является следующее:* <br>
Цепочка состоит из звеньев, которые могут быть как примитивами, так и другими цепочками. Примитивами могут быть либо промпты, LLM, utils, либо другие цепочки. Таким образом, цепочка — это, по сути, конвейер, который обрабатывает входные данные, используя определенную комбинацию примитивов. Интуитивно это можно рассматривать как «шаг», который выполняет определенный набор операций над входными данными и возвращает результат. Это может быть что угодно: от прохождения через LLM на основе промпта до применения Python-функции к тексту.

Цепочки делятся на несколько основных типов: 
* `Generic chains` - общего назначения
* `Utility chains` - выполняющие конкретную функцию (например, вычислять математические выражения)
* `Combine Documents chains` - цепочки для работы с документами <br>
и другие.

В этом ноутбуке остановимся на первых двух типах, третий специфический тип мы рассмотрим позднее.

# <center id="p2"> 💊 Generic Chains - цепочки общего назначения </center> 

<div class="alert alert-info">
    
В `LangChain` есть несколько основных типов `Generic Chains`:
* `LLM Chain` - базовый тип.
* `Transform Chain` - позволяет преобразовывать текст и применять к нему различные функции.
* `Sequentional Chain` - позволяет собирать последовательности из нескольких цепочек.
* `Router Chain` - по запросу решает куда, его перенаправить.

*Попробуем рассмотреть их всех на одном примере* 🤪

In [3]:
import os
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')

In [1]:
#!pip install --upgrade langchain langchain-openai openai tiktoken numexpr -q

In [2]:
# Для работы в колабе загрузите наш скрипт для использования ChatGPT на сервере курса!
#!wget https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py

In [3]:
# # Если используете ключ от OpenAI, запустите эту ячейку
# from langchain_openai import ChatOpenAI

# # os.environ['OPENAI_API_KEY'] = "Введите ваш OpenAI API ключ"
# os.environ['OPENAI_API_KEY'] = getpass(prompt='Введите ваш OpenAI API ключ')

# # инициализируем языковую модель
# llm = ChatOpenAI(temperature=0.0)

In [4]:
# Если используете ключ из курса, запустите эту ячейку
from utils import ChatOpenAI


#course_api_key= "Введите ваш OpenAI API ключ"
course_api_key = getpass(prompt='Введите ваш OpenAI API ключ')

# инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

Введите ваш OpenAI API ключ ········


## <center id="p3"> 🔗 `LLMChain` - базовая цепочка для общения с LLM.
  


Самой простой из этих цепочек является `LLMChain`. С ней мы уже знакомились в предыдущих уроках. 

<div class="alert alert-info">
Создадим цепочку для переписывания текста в различных стилях.

In [20]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# создадим шаблон и промпт
template = '''Перепиши текcт ниже в заданном стиле.
Текст:{output_text}

Стиль: {style}.

Результат:'''

prompt = PromptTemplate(input_variables=['output_text', 'style'], template=template) 

# создаём цепочку
style_changer_chain = LLMChain(llm=llm, prompt=prompt, output_key='final_output')   

In [21]:
text = '''Приветствуем на курсе по тюнингу языковых моделей! 
Тут мы учим, как сделать модельки еще круче и раскрыть их потенциал до максимума. 
Мы дадим вам советы по самым свежим методам обучения и настройки языковых моделей, 
а также научим практическим навыкам для решения сложных задач в обработке естественного языка.'''
style = 'Роман 18 века'

style_changer_chain.invoke({'output_text': text, 'style': style})['final_output']

'Добро пожаловать на курс по усовершенствованию языковых моделей! Здесь мы научим вас тонкостям искусства придания моделям еще большего великолепия и раскрытия их потенциала до предела. Мы предоставим вам советы по самым передовым методам обучения и настройки языковых моделей, а также научим вас практическим навыкам для решения сложнейших задач в области обработки естественного языка.'

<div class="alert alert-success">
    
Видим, что цепочка с моделью неплохо справилась с задачей.

## <center id="p4"> 🤖 `TransformChain`  - измени и передай другому

<div class="alert alert-info">
    
Теперь давайте представим, что мы получаем откуда-то сырой необработанный текст, который содержит пустые строки и много ненужных пробелов. Как вы помните, нам приходится считать количество израсходованных токенов, и не хочется их дополнительно тратить на ненужные лишние символы. К тому же такой текст выглядит неопрятно. <br>
Давайте создадим цепочку, которая будет очищать поданный в неё текст от ненужных символов. <br>
В этом нам поможет `Transform Chain`.


Для начала создадим функцию, которая будет убирать лишние пробелы и строки в исходном тексте.

In [20]:
import re

def del_spaces(inputs: dict) -> dict:
    text = inputs["text"]
    
    # заменяем пустые строки и дополнительные пробелы на один, используя регулярные выражения
    text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', text)
    text = re.sub(r'[ \t]+', ' ', text)

    return {"output_text": text}

In [21]:
# создаём преобразующую цепочку
from langchain.chains import TransformChain

text_clean_chain = TransformChain(input_variables=["text"], 
                                  output_variables=["output_text"], 
                                  transform=del_spaces)

<div class="alert alert-info">
    
**Обратите внимание**, что при создании этой цепочки, мы не подаём llm как аргумент. Можно подумать, что отсутствие LLM делает такую цепочку более слабой по сравнению с другими, но далее мы увидим, что комбинируя её с другими цепочками, можно добиться хороших результатов и сэкономить токены.<br>
Посмотрим как она работает:

In [24]:
# Возьмём наш предыдущий текст и "загрязним его"
dirty_text = '''Приветствуем на курсе по        тюнингу языковых        моделей! 

Тут мы учим, как сделать модельки еще круче и раскрыть их потенциал до максимума. 
Мы дадим вам советы по   самым свежим методам  обучения и настройки языковых                    моделей,

а также научим практическим навыкам для                     решения сложных задач в обработке естественного языка.'''

In [25]:
# Запустим, и видим, что цепочка задачу выполнила
text_clean_chain.invoke(dirty_text)['output_text']

'Приветствуем на курсе по тюнингу языковых моделей! \nТут мы учим, как сделать модельки еще круче и раскрыть их потенциал до максимума. \nМы дадим вам советы по самым свежим методам обучения и настройки языковых моделей,\nа также научим практическим навыкам для решения сложных задач в обработке естественного языка.'

<div class="alert alert-info">
    
Как вы уже наверное догадались (по названиям входных и выходных переменных), мы собираемся подать выход из `Transform Chain` в `LLMChain`. Чтобы их объединить и использовать как одну интегрированную цепочку, воспользуемся `Sequential Chain`.

## <center id="p5">  🔗➕⛓ `SequentialChain`  - последовательные цепочки

<div class="alert alert-info">
    
<img src='../images/seq_chain.jpeg' align="right" width="600" height="600" >    
    
Два основных типа `Sequential Chain`:
* `SimpleSequentialChain`: простейшая форма последовательной цепочки, где каждый шаг имеет один вход/выход, а выход одного шага является входом следующего.
* `SequentialChain`: более общая форма последовательной цепочки, допускающая несколько входов/выходов.

Воспользуемся вторым вариантом:

In [26]:
from langchain.chains import SequentialChain

In [27]:
sequential_chain = SequentialChain(chains=[text_clean_chain, style_changer_chain],
                                   input_variables=['text', 'style'],
                                   output_variables=['final_output'])

In [28]:
style = 'Rap'
print(sequential_chain.invoke({'text': dirty_text, 'style': style}))

Yo, добро пожаловать на курс по тюнингу языковых моделей!
Тут мы научим, как модельки станут еще круче, раскроем их потенциал до максимума.
Советы по самым свежим методам обучения и настройки языковых моделей - мы дадим,
Практические навыки для решения сложных задач в обработке естественного языка - научим.


<div class="alert alert-block alert-danger">
    
# <center> А теперь забываем всё, что увидели выше! 🤯
    
Разработчики `LangChain` выкатили новую универсальную удобную концепцию, встречайте:

# <center id="plcel"> **`LCEL` - LangChain Expression Language** 🤪🀄️

Что он позволяет делать?:
* декларативный способ составлять цепочки
* стандартизирует интерфейсы
* проще персонализировать различные части цепочки
* позволяет легко заменять компоненты
* поддержка потоковой, пакетной и асинхронной обработки «из коробки»
* промпты теперь стали более заметными и их можно легко изменить в соответствии с конкретными юзкейсами
    
Хватит теории, посмотрим на эту красоту в деле:

<div class="alert alert-success">

Давайте посмотрим как можно переписать цепочку из первого примера:

In [31]:
# создадим шаблон и промпт
template = '''Перепиши этот текcт в заданном стиле: {output_text}
Стиль: {style}.
Результат:'''

prompt = PromptTemplate(input_variables=['output_text', 'style'], template=template)

style_changer_chain = prompt | llm # И ВСЁ!

In [33]:
text = '''Приветствуем на курсе по тюнингу языковых моделей! 
Тут мы учим, как сделать модельки еще круче и раскрыть их потенциал до максимума. 
Мы дадим вам советы по самым свежим методам обучения и настройки языковых моделей, 
а также научим практическим навыкам для решения сложных задач в обработке естественного языка.'''
style = 'Роман 18 века'

answer = style_changer_chain.invoke({'output_text': text, 'style': style})
print(answer)

content='Добро пожаловать на курс по усовершенствованию языковых моделей, благородные господа и прекрасные дамы! Здесь мы будем обучать вас тому, как сделать ваши модели еще более великолепными и раскрыть их потенциал до предела. Мы будем делиться с вами самыми передовыми методами обучения и настройки языковых моделей, а также научим вас практическим навыкам, необходимым для решения сложных задач в обработке естественного языка. Приготовьтесь погрузиться в мир изысканных слов и изящных фраз, который ожидает вас на этом благородном пути познания.'


In [34]:
answer.content

'Добро пожаловать на курс по усовершенствованию языковых моделей, благородные господа и прекрасные дамы! Здесь мы будем обучать вас тому, как сделать ваши модели еще более великолепными и раскрыть их потенциал до предела. Мы будем делиться с вами самыми передовыми методами обучения и настройки языковых моделей, а также научим вас практическим навыкам, необходимым для решения сложных задач в обработке естественного языка. Приготовьтесь погрузиться в мир изысканных слов и изящных фраз, который ожидает вас на этом благородном пути познания.'

 <div class="alert alert-info">

 Стандартный интерфейс имеет 3 метода:
 * `stream` - возвращает ответ модели в виде итерируемого объекта
 * `invoke` - обрабатывает одиночный запрос к модели
 * `batch` - обрабатывает список запросов к модели

In [35]:
style_changer_chain.batch([{'output_text': text, 'style': style}, {'output_text': text, 'style': 'поэма'}])

[AIMessage(content='Добро пожаловать на курс по усовершенствованию языковых моделей, благородные господа и прекрасные дамы! Здесь мы будем обучать вас тому, как сделать ваши модели еще более великолепными и раскрыть их потенциал до предела. Мы будем делиться с вами самыми передовыми методами обучения и настройки языковых моделей, а также научим вас практическим навыкам, необходимым для решения сложных задач в обработке естественного языка. Приготовьтесь погрузиться в мир изысканных слов и изящных фраз, который ожидает вас на этом благородном пути познания.'),
 AIMessage(content='Приветствуем вас на курсе, где тюнинг моделей языковых\nВас ждут знания, чтобы сделать их еще круче, до максимума раскрыть\nСоветы мы дадим, самые свежие методы обучения и настройки\nПрактические навыки, сложные задачи в обработке языка естественного\n\nПогрузимся в мир, где слова станут музыкой\nГде модели языковые расцветут, как цветы весной\nСекреты мы раскроем, техники улучшения\nЧтобы язык стал красивым, с

In [36]:
# метод stream возвращает генератор
style_changer_chain.stream({'output_text': text, 'style': 'поэма'})

<generator object RunnableSequence.stream at 0x7f64f9ccce40>

In [ ]:
#for s in style_changer_chain.stream({'output_text': text, 'style': 'поэма'}):
#    print(s.content)

 <div class="alert alert-info">

Давайте теперь вспомним пример из 2-го модуля, где мы долго прикручивали `Output Parser`. Теперь использовать парсеры стало гораздо удобнее и требуется писать меньше кода! 

In [38]:
# Просто импортируем парсер и добавляем его, как ещё 1 звено
from langchain.schema.output_parser import StrOutputParser

chain_with_parser = prompt | llm | StrOutputParser() # И ВСЁ!

print(chain_with_parser.invoke({'output_text': text, 'style': style}))

Добро пожаловать на курс по усовершенствованию языковых моделей, благородные господа и прекрасные дамы! Здесь мы будем обучать вас тому, как сделать ваши модели еще более великолепными и раскрыть их потенциал до предела. Мы будем делиться с вами самыми передовыми методами обучения и настройки языковых моделей, а также научим вас практическим навыкам, необходимым для решения сложных задач в обработке естественного языка. Приготовьтесь погрузиться в мир изысканных слов и изящных фраз, который ожидает вас на этом благородном пути познания.


<div class="alert alert-success">

Вы уже, наверное, задумались как же можно переделать нашу `Sequentional Chain` с помощью `LCEL`, давайте скорее посмотрим!

In [39]:
# Немного перепишем функцию очистки
def del_spaces(inputs: dict) -> dict:
    text = inputs["output_text"]
    
    # заменяем пустые строки и дополнительные пробелы на один, используя регулярные выражения
    text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', text)
    text = re.sub(r'[ \t]+', ' ', text)

    return {"output_text": text, 'style': style}

In [40]:
seq_chain = del_spaces | prompt | llm # И больше ничего не нужно!!!

seq_chain.invoke({'output_text': dirty_text, 'style': style})

AIMessage(content='Добро пожаловать на курс по усовершенствованию языковых моделей, благородные господа и прекрасные дамы! Здесь мы будем обучать вас тому, как сделать ваши модели еще более великолепными и раскрыть их потенциал до предела. Мы будем делиться с вами самыми передовыми методами обучения и настройки языковых моделей, а также научим вас практическим навыкам, необходимым для решения сложных задач в обработке естественного языка. Приготовьтесь погрузиться в мир знаний и открытий, который открывается перед вами!')

 <div class="alert alert-info">

Стало действительно нагляднее, удобней и гораздо короче, вместо кучи ячеек кода и отдельных цепочек - всего 1 строчка!!!

Далее все примеры в ноутбуке и следующие темы курса будем показывать, используя `LCEL`. Методы, которыми пользовались ранее, тоже продолжают работать, но переведены в статус `legacy`, т.е вряд ли будут развиваться и обновляться (про них можете почитать в документации).

<div class="alert alert-success">

Рассмотрим пример с вложенными цепочками.

In [41]:
from operator import itemgetter
from langchain.prompts import ChatPromptTemplate


prompt1 = ChatPromptTemplate.from_template("В каком городе родился {person}?")
prompt2 = ChatPromptTemplate.from_template("В какой стране находится город {city}? Ответь на {language} языке.")

chain1 = prompt1 | llm | StrOutputParser()

chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | llm
    | StrOutputParser()
)

result = chain2.invoke({"person": "Колумб", "language": "немецкий"})
print(result)


Der Geburtsort von Christoph Kolumbus ist Genua, Italien.


## <center id="p6"> 🈯️ ➡️ 🇯🇵 Router Chain - определитель тем 

<div class="alert alert-info">
    
<img src='../images/rout_chain.jpeg' align="right" width="600" height="600" >   
    
Например, если у вас есть две цепочки с LLM хорошо натренированными под разные задачи, одна хорошо разбирается в цветах, а другая в футболе. То, когда вы зададите вопрос, `Router Chain` может определить тему и отправить запрос в соответствующую цепочку. <br>
Рассмотрим на примере:

In [42]:
from typing import Literal
from langchain.schema.runnable import RunnableBranch, RunnablePassthrough
from langchain.output_parsers.openai_functions import PydanticAttrOutputFunctionsParser
from langchain.pydantic_v1 import BaseModel
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

<div class="alert alert-success">
    
Так как у нас нет 2-х разных предобученых моделей под конкретную задачу, попросим ChatGPT отвечать нам, как-будто она специалист в 2 различных областях, с помощью промптов. 

In [43]:
botanist_template = """Ты очень опытный флорист и ботаник, знаешь всё о цветах, растениях.
Тебе нравится отвечать на вопросы о том, как выбирать и ухаживать за растениями. 
Ты отвечаешь так, что всё становится ясно даже начинающему цветоводу. 
Вот вопрос:
{input}"""

football_template = """Ты спортивный журналист с большим опытом, твоя основная специализация футбол.
Ты знаешь всё о футбольных командах и игроках, и очень любишь отвечать на вопросы о футболе, но кратко и по делу.
Вот вопрос:
{input}"""

In [44]:
# Создаём промпты
botanist_prompt = PromptTemplate.from_template(botanist_template)
football_prompt = PromptTemplate.from_template(football_template)

# Создаём ветки
prompt_branch = RunnableBranch(
    (lambda x: x["topic"] == "botany", botanist_prompt),
    (lambda x: x["topic"] == "football", football_prompt),
    PromptTemplate.from_template("Answer the question: {input}"),
)

In [45]:
class TopicClassifier(BaseModel):
    "Classify the topic of the user question"

    topic: Literal["botany", "football", "general"]
    "The topic of the user question. One of 'botany', 'football' or 'general'."


classifier_function = convert_pydantic_to_openai_function(TopicClassifier)

model = ChatOpenAI(course_api_key=course_api_key).bind(
    functions=[classifier_function],
    function_call={"name": "TopicClassifier"}
)
parser = PydanticAttrOutputFunctionsParser(pydantic_schema=TopicClassifier, attr_name="topic")

classifier_chain = model | parser

In [46]:
router_chain = (
    RunnablePassthrough.assign(topic=itemgetter("input") | classifier_chain)
    | prompt_branch
    | llm
    | StrOutputParser()
)

<div class="alert alert-success">

Посмотрим на результат.

In [50]:
router_chain.invoke({'input': "Какие цвета у Барселоны?"})

'Цвета Барселоны - голубой и красный.'

In [51]:
router_chain.invoke({'input':"Сколько раз в неделю поливать кактус?"})

'Кактусы являются растениями, которые приспособлены к жизни в сухих условиях, поэтому они не требуют частого полива. Обычно достаточно поливать кактус раз в 1-2 недели в течение весенне-летнего периода. Однако, частота полива может зависеть от условий окружающей среды, типа почвы и размера горшка, в котором находится кактус. Важно помнить, что перед поливом почва должна полностью высохнуть. Чтобы определить, когда полить кактус, можно провести простой тест: пальцем или деревянной палочкой проверьте влажность почвы на глубине около 2-3 см. Если она сухая, то можно полить растение. В зимний период, когда кактусы находятся в состоянии покоя, полив следует сократить до минимума, поливая раз в месяц или даже реже.'

In [52]:
router_chain.invoke({'input':"Какие цветы у Барселоны?"})

'Цвета Барселоны - голубой и красный.'

# <center id="p7"> 🛠 Utility Chains 🪝 - считают и выполняют </center>


<div class="alert alert-info">
    
Судя по названию, `Utility Chains` выполняют какую-либо утилитарную функцию: математические расчеты, отправляют запросы в интернет и работают с ответом запроса, выполняют bash-скрипты и.т.п. <br>
Их достаточно много разных, рассмотрим пример с математической цепочкой:

In [101]:
from langchain.chains import LLMMathChain

math_chain = LLMMathChain(llm=llm, verbose=True)

In [102]:
print(math_chain.invoke('Сколько будет 25 возвести в степень 0.025'))



> Entering new LLMMathChain chain...
Сколько будет 25 возвести в степень 0.025```text
25 ** 0.025
```
...numexpr.evaluate("25 ** 0.025")...

Answer: 1.0837983867343681
> Finished chain.
Answer: 1.0837983867343681


In [62]:
# Проверим
25 ** 0.025

1.0837983867343681

<div class="alert alert-info">
    
Видим, цепочка получила вопрос на естественном языке и отправила его в модель. LLM вернул код Python, который скомпилировала цепочка, чтобы дать нам ответ. <br>
Возникает вопрос: Как LLM узнала, что мы хотим, чтобы он возвращал код Python? <br>
Ответ мы можем обнаружить, если посмотрим на промпт этой цепочки:

In [56]:
print(math_chain.prompt.template)

Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
...numexpr.evaluate(text)...
```output
${{Output of running the code}}
```
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: {question}



<div class="alert alert-info">
    
Мы буквально говорим LLM, что для решения сложных математических задач он не должен пытаться решать математические задачи самостоятельно, а вместо этого должен распечатать код Python, который выполнит расчет математической задачи. Вероятно, если бы мы просто отправили запрос без какого-либо контекста, LLM попыталась бы (и не смогла) вычислить это самостоятельно. <br>
Это можно проверить... давайте попробуем! 🧐

In [65]:
prompt = PromptTemplate(input_variables=['question'], template='{question}')
chain = prompt | llm

chain.invoke({'question': 'Сколько будет 25 возвести в степень 0.025'})

AIMessage(content='25 возвести в степень 0.025 равно примерно 1.066018678.')

<div class="alert alert-success">

Неверный ответ! В этом и заключается сила промптинга. <br>
Вывод: разумно используя промптинг, мы можем заставить LLM избегать распространенных ошибок, явно программируя его на определенное поведение.

# <center id="p8"> 🧸 Выводы и заключения ✅: <br>

<div class="alert alert-info">
    
* Во всех, приведенных выше, примерах вы могли заметить, что цепочка в своей основе имеет тот или иной специально написанный промпт
* т.е. по сути можно было бы обходиться без них, проектируя новый сложный промпт под каждую задачу. 
* Но согласитесь, что с помощью цепочек `LangChain` это делать гораздо проще и удобнее. Особенно, если мы говорим про крупные сервисы. 
* Т.е. цепочки позволяют нам больше сосредоточиться на продумывании логики и функционала сервиса, а не на дизайне промптов.